In [ ]:
import pandas as pd
import numpy as np
from neuralforecast import NeuralForecast
from neuralforecast.models import Autoformer, PatchTST
import matplotlib.pyplot as plt

In [ ]:
# Load and preprocess your dataset
Y_df = pd.read_csv('../data/average_weekly_rating_messenger.csv')

# Ensure the 'ds' column is datetime
Y_df['ds'] = pd.to_datetime(Y_df.iloc[:, 0])

# Ensure the 'y' column is numeric and handle any non-numeric issues
Y_df['y'] = pd.to_numeric(Y_df.iloc[:, 1], errors='coerce')
Y_df.dropna(subset=['y'], inplace=True)  # Drop rows where 'y' is NaN after conversion

# Add unique_id as a constant value
Y_df['unique_id'] = 'rating'

# Keep only necessary columns
Y_df = Y_df[['unique_id', 'ds', 'y']]